<a href="https://colab.research.google.com/github/rjanow/Masterarbeit/blob/main/SOLYS_to_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SOLYS_to_CSV

Hier werden die Sun-Tracker Daten in eine CSV umgewandelt:

In [68]:
import os
import pandas as pd
import matplotlib as plt

from google.colab import drive

In [10]:
folder_path = f'/content/drive/My Drive/Colab_Notebooks/SOLYS_Messdaten'

In [51]:
# Funktion zum Laden aller Dateien in einem Google Drive-Ordner
def load_data_from_drive_folder(folder_path, header_lines=7, encoding = 'cp1252'):

    # Liste, um alle DataFrames zu speichern
    dfs = []

    # Durchlaufe alle Dateien im angegebenen Ordner
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Überprüfe, ob es eine Datei ist und lade sie
        if os.path.isfile(file_path):
            df = pd.read_csv(file_path, skiprows=header_lines, sep = '\t', encoding=encoding)
            dfs.append(df)

    # Kombiniere alle DataFrames
    combined_df = pd.concat(dfs, ignore_index=True)

    return combined_df

In [52]:
# Beispielaufruf der Funktion
# Ersetzen Sie 'path_to_your_folder' mit dem tatsächlichen Pfad Ihres Ordners in Google Drive
df = load_data_from_drive_folder(folder_path)

In [59]:
df

,Unit,Unnamed: 1,W/m²,W/m².1,°C,W/m².2,V,W/m².3,°C.1,°C.2,Unnamed: 10
0,1,28.12.2022 08:13:40,"14,0","16,8","9,55","17,2","0,0000","1,6","9,9","9,4",NaN
1,1,28.12.2022 08:13:50,"14,1","16,9","9,56","17,2","0,0003","1,6","9,9","9,4",NaN
2,1,28.12.2022 08:14:00,"14,2","17,0","9,54","17,4","0,0003","1,6","9,9","9,4",NaN
3,1,28.12.2022 08:14:10,"14,3","17,1","9,54","17,5","0,0003","1,6","9,9","9,4",NaN
4,1,28.12.2022 08:14:20,"14,4","17,2","9,56","17,6","0,0003","1,6","9,9","9,4",NaN
...,...,...,...,...,...,...,...,...,...,...,...
371890,1,26.01.2023 08:07:10,"27,6","30,3","-1,30","32,4","0,0000","1,4","-1,1","-1,9",NaN
371891,1,26.01.2023 08:07:20,"27,6","30,3","-1,30","32,5","0,0000","1,4","-1,1","-1,9",NaN
371892,1,26.01.2023 08:07:30,"27,7","30,4","-1,30","32,7","0,0000","1,4","-1,1","-1,9",NaN
371893,1,26.01.2023 08:07:40,"27,9","30,6","-1,30","32,9","0,0000","1,4","-1,1","-1,9",NaN


## Löschen und umbenennen der Spalten:

In [57]:
df_Solys = df.drop(columns = ['Unit', 'Unnamed: 10', ' °C', ' °C.1'], axis=1)

In [69]:
df_Solys

,Glo,Dif,Glo_SPLite,PV_Zelle,Dir,Temp
Datetime,,,,,,
28.12.2022 08:13:40,"14,0","16,8","17,2","0,0000","1,6","9,4"
28.12.2022 08:13:50,"14,1","16,9","17,2","0,0003","1,6","9,4"
28.12.2022 08:14:00,"14,2","17,0","17,4","0,0003","1,6","9,4"
28.12.2022 08:14:10,"14,3","17,1","17,5","0,0003","1,6","9,4"
28.12.2022 08:14:20,"14,4","17,2","17,6","0,0003","1,6","9,4"
...,...,...,...,...,...,...
26.01.2023 08:07:10,"27,6","30,3","32,4","0,0000","1,4","-1,9"
26.01.2023 08:07:20,"27,6","30,3","32,5","0,0000","1,4","-1,9"
26.01.2023 08:07:30,"27,7","30,4","32,7","0,0000","1,4","-1,9"


In [62]:
df_Solys.columns = ['Datetime', 'Glo', 'Dif', 'Glo_SPLite', 'PV_Zelle', 'Dir', 'Temp']

In [66]:
df_Solys.set_index('Datetime', inplace = True)

In [70]:
plt.plot(df_Solys['Glo'])

AttributeError: ignored